In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import string
from tqdm import tqdm

#Utilizando a estrutura de dicionarios, como o site reconhece que não é um ser humano usando, precisamos fazer o request usando a request que achei no site.

#Headers encontrados em NETWORK, utilizando o inspecionar, copia-se e converte CURL para PYTHON.
headers = {
    'authority': 'www.residentevildatabase.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'pt-BR,pt;q=0.9',
    'cache-control': 'max-age=0',
    'sec-ch-ua': '"Chromium";v="112", "Google Chrome";v="112", "Not:A-Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Linux"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'none',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
}

#Biblioteca BEAUTIFUL SOUP ajuda a fazer queries no HTML buscando coisas baseado em um padrao.
#Passando o objeto soup na função criada get_basic_infos.
def get_basic_infos(soup):
    
    #Utilizamos o FIND para encontrar a div com a classe page content, que foi aonde vimos aonde fica a pagina inteira com as informações do personagem.
    ems = (soup.find("div", class_="td-page-content")
               #Dentro de paragrafo
               .find("p")
               #Informações que esão dentro do EM
               .find_all("em"))
                #Ele retorna uma lista com as informações do personagem
        
    #Criando o dicionario
    data = {}
    for em in ems:
        #Transformamos em dicionario slipitando a string que pegamos com o .text, esse pega o texto dentro do elemento HTML
        chave, valor, *_ = em.text.split(":")
        #A Strip remove os espaços do final e do começo da string
        chave = chave.strip(" ").lower()
        if "nascimento" in chave:
            chave = "ano de nascimento"

        valor = valor.strip(" ").lower()
        data[chave] = valor
    
    return data

#Buscando as aparições daquele personagem
def get_aparitions(soup):
    try:
        a_tags = (soup.find("h4")
                    #Como o nivel de herarquia de h4 e p é o mesmo, utilizamos o find next
                    .find_next_sibling("p")
                    #Buscando dentro da Ancora do HTML
                    .find_all("a"))
        
        #A funcao retorna uma lista sendo os elementos cada uma das aparições.
        return {"aparicoes": [i.text for i in a_tags]}
    
    except AttributeError as err:
        return {"aparicoes": []}

def get_person_infos(url):
    
    #Procurando o nome do persongaem com o URL, atraves do split separando por barras e transformando em lista, pegamos o ultimo elemento da lista que e o nome no URL e trocamos o - por espaco
    name = url.strip("/").split("/")[-1].replace("-", " ").title()

    resp = requests.get(url, headers=headers)
    soup = BeautifulSoup(resp.text)

    data = {"nome": name}
    data.update(get_basic_infos(soup))
    data.update(get_aparitions(soup))

    return pd.Series(data)


def get_links_from_letter(letter):
    try:
        a_tags = (soup.find("h3", string=letter)
                    .find_next_sibling("p")
                    .find_all("a"))

        links = [a['href'] for a in a_tags]
        return links
    
    except AttributeError as err:
        return []


url = 'https://www.residentevildatabase.com/personagens/'

resp = requests.get(url, headers=headers)
soup = BeautifulSoup(resp.text)


all_links = []
for i in string.ascii_uppercase:
    all_links.extend(get_links_from_letter(i))


all_data = []
for i in tqdm(all_links):
    all_data.append(get_person_infos(i))


df = pd.concat([i.to_frame().T for i in all_data])
df.to_csv("dados_re.csv", index=False, sep=";")